# С помощью данной методики можно найти свободные IMSI диапазоны, а также создать файл с ключами для отправки кастомеру

In [1]:
# Загрузить данные из файлов
# Задать имена аккаунтов и файлов
#----------------------------------

import pandas as pd
from pandas import DataFrame
from os.path import join, normpath
import datetime as dt
from collections import namedtuple

### Set the variables here ###

reseller_name = 'Roamability Test RT'
customer_name = 'Orange' # to use as a file name

account_s1 = '' # Account name with S1
account_s2 = '' # Account name with S2
account_s4 = '' # Account name with S4
account_s5 = '' # Account name with S5
account_s6 = 'Orange Test Sim cards' # Account name with S6
account_s8 = '' # Account name with S8

##############################

sponsor_object = namedtuple('SponsorRange', 'sponsor test_range prod_range account_name')

s1_sponsor = sponsor_object('S1', '425019613998', '42501', account_s1)
s2_sponsor = sponsor_object('S2', '260060149991', '26006', account_s2)
s4_sponsor = sponsor_object('S4', '454030227305', '45403', account_s4)
s5_sponsor = sponsor_object('S5', '515030191000', '51503', account_s5)
s6_sponsor = sponsor_object('S6', '260036610100', '26003', account_s6)
s8_sponsor = sponsor_object('S8', '234500026400', '23450', account_s8)

downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA\SoftProfiles\Data'
result_file = 'result_file.csv'
files = ['rb_42501_25_06_20181.txt',
         'rb_42501_2_09_20181.txt',
         'rb_26006_25_06_20181.txt',
         'rb_26006_2_09_20181.txt',
         'rb_26006_30_04_2019.txt',
         'rb_42501_30_04_2019.txt',
         'rb_45403_09_07_2019.txt',
         '454030227306000_454030227308999.out',
         '454030227310000_454030227310999.out',
         '515030191000000_515030191099999.out',
         'rb_soft_191103_s1.txt',
         'rb_soft_191103_s2.txt',
         'rb_soft_191103_s4.txt',
         'rb_soft_191107_s6.txt',
         'rb_soft_191107_s8.txt',
         'rb_soft_191207_s1.txt',
         'rb_soft_191207_s2.txt',
         'rb_soft_200317_s4.txt',
         'rb_soft_200317_s8.txt',
         'rb_soft_200723_s6.txt',]

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor, s8_sponsor]

current_date = dt.datetime.now().strftime('%y%m%d')

df_imsi = DataFrame(columns = ['IMSI', 'KI', 'OPC'])

for file in files:
    df_imsi = df_imsi.append(pd.read_csv(join(downloads, file), sep=' ', engine='python'))

df_imsi.IMSI = df_imsi.IMSI.astype('int64')
df_imsi.sort_values(by = 'IMSI', inplace = True)

df_imsi.head(3)

,IMSI,KI,OPC
0,234500026400000,693E4A067EEB7F66AD9FBD5D0AA3D1E6,79EC10C3C95CED20C758681507904E6D
1,234500026400001,0C12B4359E0AF6FA8D25E39C0662C3D5,02DECFCAE2F9A6D1AED425ED810B2CA6
2,234500026400002,EC3655E2752449C2E8B1F96F79F9A2FD,79798D3604C1EDF288F237963111E7FB


In [7]:
# Из базы данных OCS выгружаем IMSI
#----------------------------------

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT
r.RESELLER_NAME
,a.ACCOUNT_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
AND ss.status = 'Active'
AND
(si.IMSI BETWEEN '260060140890000' AND '260060140899999'
OR si.IMSI BETWEEN '260060149991100' AND '260060149999999'
OR si.IMSI BETWEEN '425019613998060' AND '425019613999999'
OR si.IMSI BETWEEN '425019629920000' AND '425019629939999'
OR si.IMSI BETWEEN '425019613790000' AND '425019613799999'
OR si.IMSI BETWEEN '425019613788000' AND '425019613789999'
OR si.IMSI BETWEEN '260060149982000' AND '260060149989999'
OR si.IMSI BETWEEN '454030227305000' AND '454030227305999'
OR si.IMSI BETWEEN '454030227306000' AND '454030227308999'
OR si.IMSI BETWEEN '454030227310000' AND '454030227310999'
OR si.IMSI BETWEEN '515030191000000' AND '515030191009999'
OR si.IMSI BETWEEN '425019613960000' AND '425019613989999'
OR si.IMSI BETWEEN '260060149940000' AND '260060149969999'
OR si.IMSI BETWEEN '454030227311000' AND '454030227319999'
OR si.IMSI BETWEEN '260036610100000' AND '260036610100999'
OR si.IMSI BETWEEN '234500026400000' AND '234500026400999'
OR si.IMSI BETWEEN '425019613930000' AND '425019613959999'
OR si.IMSI BETWEEN '260060149910000' AND '260060149939999'
OR si.IMSI BETWEEN '454030227615000' AND '454030227619999'
OR si.IMSI BETWEEN '234500026425000' AND '234500026434999'
OR si.IMSI BETWEEN '260036610006000' AND '260036610015999')
'''

# OR si.IMSI BETWEEN '515030191000000' AND '515030191099999' # Full range of Smart

df_ocs_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_imsi['IMSI'] = df_ocs_imsi['IMSI'].astype('int64')
cnxn.close()
df_ocs_imsi.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI
0,1OT,Test SIM cards,Active,425019613998119
1,Clementvale,Clementvale S1 profiles,Active,425019613791002
2,Clementvale,Clementvale S1 profiles,Active,425019613791005


In [8]:
# Объединяем IMSI с ключами и данные OCS
#----------------------------------

df_ocs_imsi_ki = pd.merge(df_imsi, df_ocs_imsi, how='left', on='IMSI')
list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]
list_col = ['RESELLER_NAME', 'ACCOUNT_NAME', 'SUB_STATUS', df_ocs_imsi_ki.IMSI.astype('str').str.slice(0,12)]
df_ocs_imsi_ki_group = df_ocs_imsi_ki.groupby(list_col)['IMSI'].agg(list_agg).reset_index().sort_values(by='IMSI')

df_ocs_imsi_ki_group.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
206,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400052,234500026400999,948
225,Roamability Resellers Test,Showmac Soft,Active,234500026400,234500026400042,234500026400051,10
307,Tottolli-Tech-Mobil,Tottolli S8 Soft,Active,234500026400,234500026400002,234500026400021,20


In [4]:
# For test accounts to use the following ranges:
# S1: 425019613998*
# S2: 260060149991*
# S4: 4540302273050*
# S5: 515030191000*
# Выбрать свободные из "Roamability Invertory"

for sponsor in sponsors:
    print(f'{sponsor.sponsor} test IMSIs in OCS by Reseller:')
    display(df_ocs_imsi_ki_group[df_ocs_imsi_ki_group['IMSI'] == sponsor.test_range].sort_values('IMSI_MIN'))

S1 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
26,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,425019613998,425019613998060,425019613998064,5
127,MCN Telecom,MCN Telecom Soft SIM,Active,425019613998,425019613998065,425019613998067,3
246,Tottolli-Tech-Mobil,Test Sim cards,Active,425019613998,425019613998071,425019613998081,4
13,Dialoq - Genesisclub RT,Genesisclub Partner Soft test,Active,425019613998,425019613998072,425019613998937,18
115,Joy_telecom,Joy telecom Partner Soft test,Active,425019613998,425019613998088,425019613998929,5
315,Watchdata SG,Watchdata Partner Soft test,Active,425019613998,425019613998090,425019613998090,1
160,RedTea RT,RedTea TEST Soft-simcards account,Active,425019613998,425019613998093,425019613998491,22
215,Roamability Resellers Test,Clementvale S1 soft,Active,425019613998,425019613998113,425019613998990,6
1,1OT,Test SIM cards,Active,425019613998,425019613998118,425019613998119,2
7,DENT,S1Profiles,Active,425019613998,425019613998120,425019613998142,12


S2 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
25,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,260060149991,260060149991100,260060149991104,5
126,MCN Telecom,MCN Telecom Soft SIM,Active,260060149991,260060149991105,260060149991107,3
12,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991,260060149991110,260060149991945,18
316,Watchdata SG,Watchdata P4 Soft test,Active,260060149991,260060149991123,260060149991123,1
22,Drimsim,Drimsim S2 Soft test IMSI,Active,260060149991,260060149991124,260060149991949,12
0,1OT,Test SIM cards,Active,260060149991,260060149991135,260060149991135,1
8,DENT,S2Profiles,Active,260060149991,260060149991136,260060149991158,12
5,DENT,MultiImsiProfiles,Active,260060149991,260060149991138,260060149991478,21
222,Roamability Resellers Test,PointDume Multi,Active,260060149991,260060149991479,260060149991498,20
146,MCN Telecom,MCN Telecom Soft SIM S2,Active,260060149991,260060149991499,260060149991954,10


S4 test IMSIs in OCS by Reseller:


...

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
24,Drimsim,Drimsim S5 Soft test IMSI,Active,515030191000,515030191000000,515030191000024,25
118,Joy_telecom,Joy telecom S5 Soft,Active,515030191000,515030191000025,515030191000034,10
233,Roamability Test RT,RedTea S5 Test SIM cards,Active,515030191000,515030191000035,515030191000035,1
238,Roamability Test RT,Sigos Test Sim cards,Active,515030191000,515030191000036,515030191000036,1
147,Mondicon SIM RT,S5 soft profiles,Active,515030191000,515030191000037,515030191000056,20
297,Tottolli-Tech-Mobil,Tottolli S5 Soft,Active,515030191000,515030191000057,515030191000756,570
18,Dialoq - Genesisclub RT,Truphone S5 Soft,Active,515030191000,515030191000077,515030191000096,20
229,Roamability Resellers Test,Showmac Soft,Active,515030191000,515030191000097,515030191000106,10
231,Roamability Test RT,DID's,Active,515030191000,515030191000107,515030191000206,100
239,Roamability Test SMART Sponsor RT,Roamability Test SMART,Active,515030191000,515030191000757,515030191000760,4


S6 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
235,Roamability Test RT,Sigos Test Sim cards,Active,260036610100,260036610100000,260036610100000,1
232,Roamability Test RT,Orange Test Sim cards,Active,260036610100,260036610100001,260036610100002,2
306,Tottolli-Tech-Mobil,Tottolli S6 Soft,Active,260036610100,260036610100003,260036610100107,105
205,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610100,260036610100108,260036610100999,892


S8 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
234,Roamability Test RT,Sigos Test Sim cards,Active,234500026400,234500026400000,234500026400000,1
307,Tottolli-Tech-Mobil,Tottolli S8 Soft,Active,234500026400,234500026400002,234500026400021,20
19,Dialoq - Genesisclub RT,Truphone S8 Soft,Active,234500026400,234500026400022,234500026400041,20
225,Roamability Resellers Test,Showmac Soft,Active,234500026400,234500026400042,234500026400051,10
206,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400052,234500026400999,948


In [5]:
# For commersial usage

for sponsor in sponsors:
    print(f'{sponsor.sponsor} prod IMSIs in OCS by Reseller:')
    display(df_ocs_imsi_ki_group[(df_ocs_imsi_ki_group.IMSI.str.startswith(sponsor.prod_range, na=False))
                                 & (df_ocs_imsi_ki_group.RESELLER_NAME == 'Roamability Invertory')
                                ].sort_values('IMSI_MIN'))

S1 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
186,Roamability Invertory,Partner Soft simcards inventory,Active,425019613981,425019613981000,425019613981999,1000
187,Roamability Invertory,Partner Soft simcards inventory,Active,425019613982,425019613982000,425019613982999,1000
188,Roamability Invertory,Partner Soft simcards inventory,Active,425019613983,425019613983000,425019613983999,1000
189,Roamability Invertory,Partner Soft simcards inventory,Active,425019613984,425019613984000,425019613984999,1000
190,Roamability Invertory,Partner Soft simcards inventory,Active,425019613985,425019613985000,425019613985999,1000
191,Roamability Invertory,Partner Soft simcards inventory,Active,425019613986,425019613986000,425019613986999,1000
192,Roamability Invertory,Partner Soft simcards inventory,Active,425019613987,425019613987000,425019613987999,1000
193,Roamability Invertory,Partner Soft simcards inventory,Active,425019613988,425019613988000,425019613988999,1000
194,Roamability Invertory,Partner Soft simcards inventory,Active,425019613989,425019613989000,425019613989999,1000
195,Roamability Invertory,Partner Soft simcards inventory,Active,425019613998,425019613998531,425019613998900,370


S2 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
162,Roamability Invertory,P4 Soft simcards inventory,Active,260060149956,260060149956500,260060149956999,500
163,Roamability Invertory,P4 Soft simcards inventory,Active,260060149960,260060149960000,260060149960999,1000
164,Roamability Invertory,P4 Soft simcards inventory,Active,260060149961,260060149961000,260060149961999,1000
165,Roamability Invertory,P4 Soft simcards inventory,Active,260060149962,260060149962000,260060149962999,1000
166,Roamability Invertory,P4 Soft simcards inventory,Active,260060149963,260060149963000,260060149963999,1000
167,Roamability Invertory,P4 Soft simcards inventory,Active,260060149964,260060149964000,260060149964999,1000
168,Roamability Invertory,P4 Soft simcards inventory,Active,260060149965,260060149965000,260060149965999,1000
169,Roamability Invertory,P4 Soft simcards inventory,Active,260060149966,260060149966000,260060149966999,1000
170,Roamability Invertory,P4 Soft simcards inventory,Active,260060149967,260060149967000,260060149967999,1000
171,Roamability Invertory,P4 Soft simcards inventory,Active,260060149968,260060149968000,260060149968999,1000


S4 prod IMSIs in OCS by Reseller:


...

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
213,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191000,515030191000761,515030191000999,239
214,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191001,515030191001300,515030191001999,700
215,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191008,515030191008000,515030191008999,1000
216,Roamability Invertory,Smart Soft Profiles inventory,Active,515030191009,515030191009000,515030191009999,1000


S6 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
197,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610006,260036610006000,260036610006999,1000
198,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610007,260036610007000,260036610007999,1000
199,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610008,260036610008000,260036610008999,1000
200,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610009,260036610009000,260036610009999,1000
201,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610010,260036610010000,260036610010999,1000
202,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610011,260036610011000,260036610011999,1000
203,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610012,260036610012000,260036610012999,1000
204,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610013,260036610013000,260036610013999,1000
205,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610014,260036610014000,260036610014999,1000
206,Roamability Invertory,S6 Soft Profiles inventory,Active,260036610015,260036610015000,260036610015999,1000


S8 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
208,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026400,234500026400052,234500026400999,948
209,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026426,234500026426550,234500026426999,450
210,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026432,234500026432000,234500026432999,1000
211,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026433,234500026433000,234500026433999,1000
212,Roamability Invertory,S8 Soft Profiles inventory,Active,234500026434,234500026434000,234500026434999,1000


In [9]:
# Записать выбранные диапазоны IMSI
# Проверить выбранные IMSI на принадлежность к "Roamability Invertory"

# После переноса IMSI в аккаунты Кастомеров проверить выбранные IMSI и диапазоны
# (предварительно перезагрузив данные из OCS)

imsi_range_object = namedtuple('ImsiRange', 'sponsor imsi_list')

def imsi_list(range_start, range_end):
    return [str(imsi) for imsi in range(range_start, range_end+1,1)]

### Set the IMSI ranges here ###

s1_imsi_range = imsi_range_object('S1', imsi_list(0, 0))
s2_imsi_range = imsi_range_object('S2', imsi_list(0, 0))
s4_imsi_range = imsi_range_object('S4', imsi_list(0, 0))
s5_imsi_range = imsi_range_object('S5', imsi_list(0, 0))
s6_imsi_range = imsi_range_object('S6', imsi_list(260036610100108, 260036610100108+3))
s8_imsi_range = imsi_range_object('S8', imsi_list(0, 0))

#################################

imsi_ranges = [s1_imsi_range, s2_imsi_range, s4_imsi_range, s5_imsi_range, s6_imsi_range, s8_imsi_range]

for imsi_range in imsi_ranges:
    if imsi_range.imsi_list:
        display(df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(imsi_range.imsi_list)].groupby(['RESELLER_NAME',
                                                                                'ACCOUNT_NAME','SUB_STATUS']).count())

,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,
Roamability Test RT,Orange Test Sim cards,Active,4


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


In [10]:
# Выгружаем файлы с ключами для диапазонов IMSI
#----------------------------------

for sponsor, imsi_range in zip(sponsors, imsi_ranges):
    if sponsor.account_name and imsi_range.imsi_list:
        df_ocs_imsi_ki.loc[(df_ocs_imsi_ki.RESELLER_NAME == reseller_name)
                           & (df_ocs_imsi_ki.ACCOUNT_NAME == sponsor.account_name)
                           & (df_ocs_imsi_ki.IMSI.astype('str').isin(imsi_range.imsi_list)),
                           ['IMSI', 'KI', 'OPC']].\
        to_csv(join(downloads, f'{sponsor.sponsor}_{customer_name}_{current_date}.txt'), index=False)
        print(f'{sponsor.sponsor}_{customer_name}_{current_date}.txt')

S6_Orange_200728.txt


### Далее шифруем и отправляем
----------------------------------
1. Упаковать оба файла в zip архив.
2. Выполнить экспорт открытого ключа Кастомера (дважды кликнуть по файлу ключа).
3. В папке с архивом по пр.кл. на zip архив и выбрать "Зашифровать"
4. Зашифровать своим ключем *_FOR_KEYS
5. Отправить кастомеру. Свой ключ отправлять не нужно.